# Hashing Trick을 사용한 단어 임베딩
- corpus 전체 단어 20개.
- vocabulary 크기 10 한정.
- embedding size, 즉, 단어의 latent feature는 8로 설정.

In [3]:
# 모듈 불러 오기
import tensorflow as tf
from tensorflow.keras.preprocessing.text import hashing_trick
from tensorflow.keras.layers import Input, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

In [5]:
# sample 문장
sentences =  ['너 오늘 이뻐 보인다',
              '나는 오늘 기분이 더러워',
              '끝내주는데, 좋은 일이 있나봐',
              '나 좋은 일이 생겼어',
              '아 오늘 진짜 짜증나',
              '환상적인데, 정말 좋은거 같아']
labels = [[1], [0], [1], [1], [0], [1]]

* collision 문제가 발생하고 있음을 알 수 있다. `오늘`과 `더러워`가 같은 단어로 나온다.

In [8]:
# hash 테이블로 문서 수치화
VOCAB_SIZE = int(input('vocabulary 크기 설정: '))
sequences = [hashing_trick(s, VOCAB_SIZE) for s in sentences]
sequences = np.array(sequences)
labels = np.array(labels)
print(sequences)

vocabulary 크기 설정: 10
[[8 1 9 7]
 [5 1 6 1]
 [4 6 3 1]
 [6 6 3 7]
 [2 1 5 3]
 [8 1 7 6]]


* 하드 코딩 요소를 줄이자!
* 패딩 요소 없기 때문에 `+1` 해서 잡아 준다.
* `tf.reduce_mean` 사용해서 문장 자체를 하나의 수치로 임베딩시켜 버렸다.

In [9]:
# 모델 파라미터 설정
EMBEDDING_SIZE = int(input('임베딩 차원 설정: '))

# 모델 네트워크 구성
X_input = Input(batch_shape=(None, sequences.shape[1]))
X_embed = Embedding(input_dim=VOCAB_SIZE+1,
                    output_dim=EMBEDDING_SIZE)(X_input)
X_embed_input = tf.reduce_mean(X_embed, axis=-1) # 가장 안에 있는 값들을 평균
X_hidden = Dense(128, activation=tf.nn.relu)(X_embed_input)
y_output = Dense(1, activation='sigmoid')(X_hidden)

# 모델 구성
model = Model(X_input, y_output)
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01))
print("========== 모델 전체 구조 ==========")
print(model.summary())

# 학습
model.fit(sequences, labels, epochs=100)



임베딩 차원 설정: 8
========== 모델 전체 구조 ==========
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
embedding (Embedding)        (None, 4, 8)              88        
_________________________________________________________________
tf_op_layer_Mean (TensorFlow [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 128)               640       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 857
Trainable params: 857
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 0.

In [11]:
# 예측
pred = model.predict(sequences)
print("추정 결과")
print(np.round(pred, 0))

추정 결과
[[1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]]
